<a href="https://colab.research.google.com/github/vkrisvasan/GenAIAgent/blob/main/AgentEx2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Building AI Agents With Crew AI using Google Gemini, Groq, LLama3
#CrewAI is an open-source framework that helps you build your own autonomous AI agents



In [1]:
pip install crewai langchain-google-genai crewai_tools

In [21]:
import os
import getpass
credential_names = ["GOOGLE_API_KEY","SERPER_API_KEY"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

In [22]:
modelToBeused = "gemini-1.5-flash"

In [23]:
from crewai import Agent
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model=modelToBeused,verbose=True, temperature=0.5,google_api_key=os.getenv("GOOGLE_API_KEY"))

In [24]:
researcher= Agent(
role="Senior Researcher",
goal="""Uncover groundbreaking topics
in {topic}""",
backstory=("""Driven by curiosity, you are at the forefront of Innovation sharin knowledge and news that would change the world""")
,tools=[],llm=llm,allow_delegation=True,verbose=True,memory=True,)

In [25]:

writer= Agent(
role="Writer",
goal="Narrate compelling tech stories about {topic}",
backstory=("""Your expertise lies in breaking down complex topics into simpler a digestable knowledge, you specialize in
captivativing audience attention with yo writing and create engaging narratives and stories""")
,tools=[],llm=llm,allow_delegation=False,verbose=True,memory=True)

In [26]:
from crewai_tools import SerperDevTool
os.environ["SERPER_API_KEY"]= os.getenv("SERPER_API_KEY")
tool= SerperDevTool()

In [27]:
from crewai import Task

research_task= Task(description=(
"Identify the next big trend in {topic}."
"Focus on identifying pros and cons and the overall narrative."
                          "Your final report should clearly articulate the key points,"
                                  "its market opportunities, and potential risks."
),
expected_output='A comprehensive 3 paragraphs long report on the latest AI trend',
tools=[tool],
agent=researcher)

write_task = Task(description=(
"Compose an insightful article on {topic}."
"Focus on the latest trends and how it's impacting the industry."
        "This article should be easy to understand, engaging, and positive."
),
expected_output='A 4 paragraph article on {topic} advancements formatted as markdown',
                  tools=[tool],
agent=writer,
async_execution=False,
output_file='new-blog-post.md' # output file containing final draft
)

In [28]:
from crewai import Crew,Process

crew= Crew(agents=[researcher,writer],
           tasks=[research_task,write_task],process= Process.sequential )


In [29]:
result= crew.kickoff(inputs={"topic":"AI in healthcare"})

print(result)



> Entering new CrewAgentExecutor chain...
Thought: I need to find out what the latest trends in AI in healthcare are. 

Action: Search the internet
Action Input: { "search_query": "AI trends in healthcare" } 
 

{'message': 'Unauthorized.', 'statusCode': 403}

Thought: I need to find out what the latest trends in AI in healthcare are. 

Action: Search the internet
Action Input: { "search_query": "AI trends in healthcare" }  

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to find out what the latest trends in AI in healthcare are. 

Action: Search the internet
Action Input: { "search_query": "latest AI trends in healthcare" }  

{'message': 'Unauthorized.', 'statusCode': 403}

Thought: I need to find out what the latest trends in AI in healthcare are. 

Action: Delegate work to co-worker
Action Input: { "task": "Research the latest trends in AI in healthcare and provide a comprehensive report on one of them, in